# 04 - FastAPI Integration

Use Poldantic to bridge Pydantic models and Polars DataFrames in FastAPI.

In [1]:
from pydantic import BaseModel
import polars as pl
from poldantic import to_polars_schema, to_pydantic_model

## 1. Define Input Model

In [2]:
class UserInput(BaseModel):
    id: int
    name: str
    email: str
    age: int

print('Input model defined')

Input model defined


## 2. Create Polars Schema

In [3]:
schema = to_polars_schema(UserInput)
print('Polars schema:', schema)

Polars schema: {'id': Int64, 'name': String, 'email': String, 'age': Int64}


## 3. Generate Response Model

In [4]:
# Create response model with same schema
UserOutput = to_pydantic_model(schema, 'UserOutput', force_optional=False)

print('Response model created')
print('Fields:', list(UserOutput.model_fields.keys()))

Response model created
Fields: ['id', 'name', 'email', 'age']


## 4. Simulate API Endpoint Logic

In [5]:
# Simulate database data in Polars
users_df = pl.DataFrame([
    {'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'age': 30},
    {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'age': 25},
    {'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'age': 35},
], schema=schema)

print('DataFrame with', users_df.shape[0], 'users')
print(users_df)

DataFrame with 3 users
shape: (3, 4)
┌─────┬─────────┬─────────────────────┬─────┐
│ id  ┆ name    ┆ email               ┆ age │
│ --- ┆ ---     ┆ ---                 ┆ --- │
│ i64 ┆ str     ┆ str                 ┆ i64 │
╞═════╪═════════╪═════════════════════╪═════╡
│ 1   ┆ Alice   ┆ alice@example.com   ┆ 30  │
│ 2   ┆ Bob     ┆ bob@example.com     ┆ 25  │
│ 3   ┆ Charlie ┆ charlie@example.com ┆ 35  │
└─────┴─────────┴─────────────────────┴─────┘


In [6]:
# Convert to API response format
response_data = users_df.to_dicts()
print('\nResponse data:')
print(response_data)

# Validate with response model
validated_users = [UserOutput(**user) for user in response_data]
print('\nValidated responses:')
for user in validated_users:
    print(f'  {user}')


Response data:
[{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'age': 30}, {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'age': 25}, {'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'age': 35}]

Validated responses:
  id=1 name='Alice' email='alice@example.com' age=30
  id=2 name='Bob' email='bob@example.com' age=25
  id=3 name='Charlie' email='charlie@example.com' age=35


## 5. Complete FastAPI Example (Pseudo-code)

```python
from fastapi import FastAPI

app = FastAPI()

@app.get('/users', response_model=list[UserOutput])
def list_users():
    # Get data from Polars DataFrame
    df = users_df.filter(pl.col('age') >= 25)
    return df.to_dicts()

@app.post('/users', response_model=UserOutput)
def create_user(user: UserInput):
    # Add to DataFrame
    new_row = pl.DataFrame([user.model_dump()], schema=schema)
    # ... append to database
    return user
```